In [2]:
import os
import snowflake.snowpark.functions as F

In [3]:
path_utils = os.path.join(os.getcwd(),'Utils','utils.ipynb')
%run $path_utils

####################################################
Available packages:
    from snowflake.snowpark.session import Session
    from snowflake.snowpark import functions as F
    from snowflake.snowpark.types import *
    from datetime import datetime

    import configparser
    import os

####################################################

Available functions, for details use help(<function_name>:
    get_snowpark_session
    set_schema
    read_csv_from_stage
    create_hz_dim
    get_field_list



In [4]:
session = get_snowpark_session()
set_schema('HZ_CLEAR_STRATEGY')

[Row(status='Statement executed successfully.')]

## DIM_CALENDAR

In [152]:
query_dim_calendar = """
    select distinct
        cast(d.date_of_game as date) as "date",
        year(cast(d.date_of_game as date)) as year,
        month(cast(d.date_of_game as date)) as month,
        day(cast(d.date_of_game as date)) as day,
        cast(to_char(cast(d.date_of_game as date),'%Y-%m') as varchar(7)) as season,
        cast(case when month(cast(d.date_of_game as date)) <= 3 then concat(year(cast(d.date_of_game as date)),'_01')
            when month(cast(d.date_of_game as date)) <= 6 then concat(year(cast(d.date_of_game as date)),'_02')
            when month(cast(d.date_of_game as date)) <= 9 then concat(year(cast(d.date_of_game as date)),'_03')
            else concat(year(cast(d.date_of_game as date)),'_04') end as varchar(7))  as year_quarter,
        case when month(cast(d.date_of_game as date)) <= 3 then '01'
            when month(cast(d.date_of_game as date)) <= 6 then '02'
            when month(cast(d.date_of_game as date)) <= 9 then '03' else'04' end as quarter,
        weekofyear(cast(d.date_of_game as date)) as week_of_year,
        dayofweek(cast(d.date_of_game as date)) as day_of_week,
        case when dayofweekiso(cast(d.date_of_game as date)) in (6,7) then 1 else 0 end as weekend    
    from pz_clear_strategy.football_data d
    where d.date_of_game is not null
    order by cast(d.date_of_game as date)
"""

df_dim_calendar = session.sql(query_dim_calendar)
df_dim_calendar.write.mode("overwrite").saveAsTable('dim_calendar')


## DIM_AREA_OF_SHOT

In [165]:
table_name = 'dim_area_of_shot'

query_new = f"""select distinct    
                    upper(d.area_of_shot) as area_of_shot,
                    d.path,
                    d.load_at
                from pz_clear_strategy.football_data d
                where d.area_of_shot is not null"""

query_append = f"""select distinct 
                        upper(d.area_of_shot) as area_of_shot,
                        d.path,
                        d.load_at
                    from pz_clear_strategy.football_data d
                    where not exists (select 1 from hz_clear_strategy.{table_name} aos
                                            where aos.area_of_shot = upper(d.area_of_shot))
                    and d.area_of_shot is not null
                    and d.load_at >= (select max(load_at) from {table_name})"""

create_hz_dim(table_name, query_new, query_append)

Table: dim_area_of_shot will be created
------------------------------------------------------
|"status"                                            |
------------------------------------------------------
|SEQ_DIM_AREA_OF_SHOT already exists, statement ...  |
------------------------------------------------------

Execution finished, dim_area_of_shot created


## DIM_SHOT_BASICS

In [154]:
table_name = 'dim_shot_basics'

query_new = f"""select distinct    
                    d.shot_basics,
                    d.path,
                    d.load_at
                from pz_clear_strategy.football_data d
                where d.shot_basics is not null"""

query_append = f"""select distinct    
                        UPPER(d.shot_basics) AS shot_basics,
                        d.path,
                        d.load_at
                    from pz_clear_strategy.football_data d
                    where d.shot_basics is not null
                    and d.load_at >= (select max(load_at) from {table_name})
                    and not exists (select 1 from hz_clear_strategy.{table_name} dsb
                                            where dsb.shot_basics = UPPER(d.shot_basics))"""

create_hz_dim(table_name, query_new, query_append)

Table: dim_shot_basics will be created
------------------------------------------------------
|"status"                                            |
------------------------------------------------------
|SEQ_DIM_SHOT_BASICS already exists, statement s...  |
------------------------------------------------------

Execution finished, dim_shot_basics created


## DIM_COMBINED_SHOT_TYPES

In [155]:
table_name = 'dim_combined_shot_types'

query_new = f"""select distinct
                upper(d.type_of_combined_shot) as combined_shot_type,
                d.path,
                d.load_at
            from pz_clear_strategy.football_data d
            where d.type_of_combined_shot is not null"""

query_append = f"""select distinct
                    upper(d.type_of_combined_shot) as combined_shot_type,
                    d.path,
                    d.load_at
                from pz_clear_strategy.football_data d
                where d.type_of_combined_shot is not null
                  and d.load_at >= (select max(load_at) from {table_name})
                and not exists (select 1 from hz_clear_strategy.{table_name} cst
                                        where cst.combined_shot_type = UPPER(d.type_of_combined_shot))
"""
create_hz_dim(table_name, query_new, query_append)

Table: dim_combined_shot_types will be created
------------------------------------------------------
|"status"                                            |
------------------------------------------------------
|SEQ_DIM_COMBINED_SHOT_TYPES already exists, sta...  |
------------------------------------------------------

Execution finished, dim_combined_shot_types created


## DIM_TEAM

In [12]:
table_name = 'dim_team'

query_new = f"""select 
                nba.short_name as team_short_name,
                UPPER(nba.team_name) as team_name,
                d.path,
                d.load_at
            from pz_clear_strategy.football_data d

            left join pz_clear_strategy.nba_teams nba
                on nba.short_name = trim(substr(d.home,1,charindex(' ',d.home)))

            where d.home is not null

            union

            select 
                nba.short_name as team_short_name,
                UPPER(nba.team_name) as team_name,
                d.path,
                d.load_at
            from pz_clear_strategy.football_data d

            left join pz_clear_strategy.nba_teams nba
                on nba.short_name = trim(substr(d.home,charindex(' ',d.home,charindex(' ',d.home)+1),len(d.home)))

            where d.home is not null"""

query_append = f"""select * from (
                                select 
                                    nba.short_name as team_short_name,
                                    UPPER(nba.team_name) as team_name,
                                    d.path,
                                    d.load_at as load_at
                                from pz_clear_strategy.football_data d

                                left join pz_clear_strategy.nba_teams nba
                                    on nba.short_name = trim(substr(d.home,1,charindex(' ',d.home)))

                                where d.home is not null

                                union

                                select 
                                    nba.short_name as team_short_name,
                                    UPPER(nba.team_name) as team_name,
                                    d.path,
                                    d.load_at as load_at
                                from pz_clear_strategy.football_data d

                                left join pz_clear_strategy.nba_teams nba
                                    on nba.short_name = trim(substr(d.home,charindex(' ',d.home,charindex(' ',d.home)+1),len(d.home)))

                                where d.home is not null
                                and d.load_at >= (select max(load_at) from {table_name})
                    ) t                               
                    where team_name is not null
                    and not exists (select 1 from hz_clear_strategy.{table_name} dteam
                                            where dteam.team_short_name = t.team_short_name)

                    union
                    
                    select
                        nba_game.team_abbreviation_home as team_short_name,
                        upper(nba_game.team_name_home) as team_name,
                        nba_game.path,
                        nba_game.load_at
                    from pz_clear_strategy.nba_game_data nba_game
                    where not exists (select 1 from hz_clear_strategy.dim_team dteam
                                            where dteam.team_short_name = upper(nba_game.team_name_home))

                    union
                    
                    select
                        nba_game.team_abbreviation_away as team_short_name,
                        upper(nba_game.team_name_away) as team_name,
                        nba_game.path,
                        nba_game.load_at
                    from pz_clear_strategy.nba_game_data nba_game
                    where not exists (select 1 from hz_clear_strategy.dim_team dteam
                                            where dteam.team_short_name = upper(nba_game.team_name_away))
"""
create_hz_dim(table_name, query_new, query_append)

--------------------------------------------------------------------------------------------------------------------
|"ID_DIM_TEAM"  |"TEAM_SHORT_NAME"  |"TEAM_NAME"        |"PATH"                       |"LOAD_AT"                   |
--------------------------------------------------------------------------------------------------------------------
|158            |MANU               |MANCHESTER UNITED  |raw_data_stage/yds_data.csv  |2023-06-10 18:37:44.517000  |
--------------------------------------------------------------------------------------------------------------------

Table: dim_team alread exists
Execution finished, dim_team appended


## DIM_RANGE_OF_SHOT

In [158]:
table_name = 'dim_range_of_shot'

query_new = f"""select distinct    
                    upper(d.range_of_shot) as range_of_shot,
                    d.path,
                    d.load_at
                from pz_clear_strategy.football_data d
                where d.range_of_shot is not null"""

query_append = f"""select distinct    
                    upper(d.range_of_shot) as range_of_shot,
                    d.path,
                    d.load_at
                from pz_clear_strategy.football_data d
                where d.range_of_shot is not null
                and d.load_at >= (select max(load_at) from {table_name})
                and not exists (select 1 from hz_clear_strategy.{table_name} ros
                                        where ros.range_of_shot = UPPER(d.range_of_shot))
"""
create_hz_dim(table_name, query_new, query_append)

-------------------------------------------------------------------------
|"ID_DIM_RANGE_OF_SHOT"  |"RANGE_OF_SHOT"  |"LOAD_AT"                   |
-------------------------------------------------------------------------
|1                       |16-24 FT.        |2023-06-10 20:24:28.085000  |
-------------------------------------------------------------------------

Table: dim_range_of_shot alread exists
Table: dim_range_of_shot will be created
------------------------------------------------------
|"status"                                            |
------------------------------------------------------
|SEQ_DIM_RANGE_OF_SHOT already exists, statement...  |
------------------------------------------------------

Execution finished, dim_range_of_shot created


## DIM_SHOT_TYPES

In [159]:
table_name = 'dim_shot_types'

query_new = f"""select distinct
                    upper(d.type_of_shot) as shot_type,
                    d.path,
                    d.load_at
                from pz_clear_strategy.football_data d
                where d.type_of_shot is not null"""

query_append = f"""select distinct    
                    upper(d.type_of_shot) as shot_type,
                    d.path,
                    d.load_at
                from pz_clear_strategy.football_data d
                where d.type_of_shot is not null
                and d.load_at >= (select max(load_at) from {table_name})
                and not exists (select 1 from hz_clear_strategy.{table_name} st
                                        where st.type_of_shot = UPPER(d.type_of_shot))
"""
create_hz_dim(table_name, query_new, query_append)

Table: dim_shot_types will be created
------------------------------------------------------
|"status"                                            |
------------------------------------------------------
|SEQ_DIM_SHOT_TYPES already exists, statement su...  |
------------------------------------------------------

Execution finished, dim_shot_types created


## DIM_STADIUM (External)

In [41]:
table_name = 'dim_stadium'

query_new = f"""select
                    lat_long,
                    upper(stadium_name) as stadium_name,
                    upper(city) as city,
                    upper("state") as state ,
                    load_at               
                from pz_clear_strategy.nba_stadiums"""

query_append = f"""select
                        lat_long,
                        upper(stadium_name) as stadium_name,
                        upper(city) as city,
                        upper("state") as state ,
                        load_at               
                    from pz_clear_strategy.nba_stadiums d
                    where d.load_at >= (select max(load_at) from {table_name})
                    and not exists (select 1 from hz_clear_strategy.{table_name} st
                                            where st.stadium_name = UPPER(d.stadium_name))
"""
create_hz_dim(table_name, query_new, query_append)

Table: dim_stadium will be created
--------------------------------------------------
|"status"                                        |
--------------------------------------------------
|Sequence SEQ_DIM_STADIUM successfully created.  |
--------------------------------------------------

Execution finished, dim_stadium created


## FT_SHOTS

In [6]:
session.sql(f'CREATE SEQUENCE IF NOT EXISTS hz_clear_strategy.seq_ft_shots START 1 INCREMENT 1;').show()

query_shots = f"""
        select 
            cast(d.match_id as int) as match_id,
            hz_clear_strategy.seq_ft_shots.NEXTVAL as id_ft_shots,
            cast(d.shot_id_number as int) as shot_id,        
            st.id_dim_shot_types,
            cst.id_dim_combined_shot_types,
            coalesce(dt.id_dim_team, lag(id_dim_team, 1) ignore nulls over (partition by match_id order by shot_id),
                                     lead(id_dim_team,1) ignore nulls over (partition by match_id order by shot_id)) as id_dim_team,
            ros.id_dim_range_of_shot,   
            b.id_dim_shot_basics,
            aos.id_dim_area_of_shot,
            cast(location_x as int) as location_x,
            cast(location_y as int) as location_y,
            cast(coalesce(d.remaining_min,d.remaining_min2) as int) as remaining_min_int,
            cast(coalesce(d.remaining_min2,d.remaining_min) as float) as remaining_min_float,
            cast(coalesce(d.power_of_shot,d.power_of_shot3) as int) as power_of_shot_int,
            cast(d.power_of_shot3 as float) as power_of_shot_float,
            cast(coalesce(d.remaining_sec,d.remaining_sec5) as int) as remaining_sec_int,
            cast(coalesce(d.remaining_sec5,d.remaining_sec) as float) as remaining_sec_float,
            cast(coalesce(d.distance_of_shot,d.distance_of_shot6) as int) as distance_of_shot_int,
            cast(coalesce(d.distance_of_shot6,d.distance_of_shot) as float) as distance_of_shot_float,
            cast(d.is_goal as int) as is_goal,
            d.path,
            d.load_at
        from pz_clear_strategy.football_data d
    
        left join cz_clear_strategy.dim_shot_basics b
        on b.shot_basics = d.shot_basics
    
        left join cz_clear_strategy.dim_team dt
        on dt.team_name = upper(d.team_name)
    
        left join cz_clear_strategy.dim_area_of_shot aos
        on aos.area_of_shot = upper(d.area_of_shot)
    
        left join cz_clear_strategy.dim_range_of_shot ros
        on ros.range_of_shot = upper(d.range_of_shot)
    
        left join cz_clear_strategy.dim_shot_types st
        on st.shot_type = upper(d.type_of_shot)
    
        left join cz_clear_strategy.dim_combined_shot_types cst
        on cst.combined_shot_type = upper(d.type_of_combined_shot)
        
        where d.load_at >= (select max(load_at) from hz_clear_strategy.ft_shots)

        order by cast(d.match_id as int)        
"""

df_shots = session.sql(query_shots)
df_shots.write.mode("Append").saveAsTable('ft_shots')

-----------------------------------------------------
|"status"                                           |
-----------------------------------------------------
|SEQ_FT_SHOTS already exists, statement succeeded.  |
-----------------------------------------------------



## FT_MATCH

In [20]:
query_matchs = f"""
    with w_shots as (
    select  
        cast(d.match_id as int) as match_id,
        cast(coalesce(game_season, lag(game_season, 1) ignore nulls over (partition by match_id order by match_id),
                                lead(game_season,1) ignore nulls over (partition by match_id order by match_id)) as varchar(7)) as game_season,
        coalesce(d.home, lag(d.home, 1) ignore nulls over (partition by match_id order by match_id),
                                lead(d.home,1) ignore nulls over (partition by match_id order by match_id)) as home,
        cast(coalesce(date_of_game, lag(date_of_game, 1) ignore nulls over (partition by match_id order by match_id),
                                    lead(date_of_game,1) ignore nulls over (partition by match_id order by match_id)) as date) as date_of_game,            
        coalesce(d.lat_long, lag(d.lat_long, 1) ignore nulls over (partition by match_id order by match_id),
                            lead(d.lat_long,1) ignore nulls over (partition by match_id order by match_id)) as lat_long,        
        cast(coalesce(is_goal,'0') as int) as is_goal,
        d.path,
        d.load_at
    from pz_clear_strategy.football_data d
    )
        
    select * from (
        select 
            w.match_id,
            w.game_season as game_season,
            coalesce(w.date_of_game, cast(nba.game_date as date)) as date_of_game,
            coalesce(dth.id_dim_team,dth2.id_dim_team) as id_dim_team_home,
            coalesce(dta.id_dim_team,dta2.id_dim_team) as id_dim_team_away,
            ds.id_dim_stadium,
            count(1) as number_of_shots,
            sum(is_goal) as number_of_goals,
            cast(nba.pts_home as int) as ext_pts_home,
            cast(nba.pts_away as int) as ext_pts_away,
            nba.wl_home as ext_wl_home,
            nba.wl_away as ext_wl_away,
            w.path,
            sysdate() as load_at,
            row_number() over (partition by w.match_id order by w.load_at desc) as rnk
        from w_shots w

        left join hz_clear_strategy.dim_stadium ds
            on ds.lat_long = replace(w.lat_long,',','')
                
        left join pz_clear_strategy.nba_game_data nba
            on cast(nba.game_id as int) = w.match_id
        
        left join cz_clear_strategy.dim_team dth
            on dth.team_short_name = nba.team_abbreviation_home
           and dth.team_name = upper(nba.team_name_home)

        left join cz_clear_strategy.dim_team dta
            on dta.team_short_name = nba.team_abbreviation_away
           and dta.team_name = upper(nba.team_name_away)

        left join cz_clear_strategy.dim_team dth2
            on dth2.team_short_name = trim(substr(w.home,1,charindex(' ',w.home)))

        left join cz_clear_strategy.dim_team dta2
            on dta2.team_short_name = trim(substr(w.home,charindex(' ',w.home,charindex(' ',w.home)+1),len(w.home)))

        --where w.load_at >= (select max(load_at) from hz_clear_strategy.ft_match)    
                
        group by w.match_id,
            w.game_season,
            coalesce(w.date_of_game, cast(nba.game_date as date)),
            coalesce(dth.id_dim_team,dth2.id_dim_team),
            coalesce(dta.id_dim_team,dta2.id_dim_team),
            ds.id_dim_stadium,
            cast(nba.pts_home as int),
            cast(nba.pts_away as int),
            nba.wl_home,
            nba.wl_away,
            w.path,
            w.load_at
   ) where rnk = 1
"""

df_matchs = session.sql(query_matchs)
df_matchs = df_matchs.drop("rnk")

df_matchs.write.mode("append").saveAsTable('ft_match')

In [21]:
session.close()